In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.integrate as si
import numpy.linalg as npla
import scipy.linalg
from scipy.linalg import solve_banded


import pandas as pd
from scipy.sparse.csc import csc_matrix

In [ ]:
#Wigner/Husini function implementation
#Particle on ring Dimensionless


#variables
Omeg0=1.0
Omeg1=1.0
eps=0.07
ntild=10#ntild = 1 is quantum, ntild>>1 is classical
dt=0.01
nsteps=300
isteps=300

t0=0
theta0=0+0.1
thetamin=-np.pi
thetamax=np.pi
k=1j

#dependent variables
dtheta=(2*thetamax)/isteps
sigma=0.5

#initial arrays 
theta=np.zeros(isteps)
drv=np.zeros(nsteps+1)
t=np.zeros(nsteps+1)
   
for n in range(nsteps+1):#t,drv array, drv is drive force
    tn=t0+(n)*dt
    t[n]=tn
    drvn=np.cos(Omeg1*tn)
    drv[n]=drvn

Psi=np.zeros(isteps,np.complex)#wave packet array

for i in range(0,isteps):#theta,initial wave packet array T
    thetai=thetamin+(i)*dtheta
    theta[i]=thetai
    Psii=np.exp((-0.0*k*thetai)+(-1)*((thetai-theta0)**2)/(sigma)**2)#this will be the initial function input
    Psi[i]=Psii
    
#probibility here, to normalize
Total=np.sum(np.abs(Psi)**2)*dtheta
Psi=Psi/(np.sqrt(Total))
#make array of arrays of T
Psin=[]
Psin.append(Psi)

#definitions of coeffients
def a(drv,theta):
    a1=((1/(ntild*2*2*(dtheta)**2))-((k*eps*(Omeg0/Omeg1)*drv[n])/(dtheta)))
    return a1

def CrankNicolson(Psi,n):#Crank nicolson solver with periodic boundary conditions
    RHS=np.zeros(isteps,np.complex)#RHS
    for i in range(-1,isteps-1):
        temp1=-a(drv,theta[i])
        temp2=1+2*a(drv,theta[i])
        temp3=-c(drv,theta[i])
        RHS[i]=temp1*Psi[i+1]+temp2*Psi[i]+temp3*Psi[i-1]
    D=np.matrix(RHS).T#made matrix for the RHS
    d=np.zeros((3,isteps),np.complex)#LHS
    for i in range(-1,isteps-1):#make the banded matrix
        d[1,i]=1-2*a(drv,theta[i])
        d[2,i]=a(drv,theta[i])
        d[0,i]=a(drv,theta[i])
    d=np.matrix(d)#made matrix for LHS
    Psi1=scipy.linalg.solve_banded((1,1),d,D)
    return Psi1

#plt.plot(theta,np.real(T))# plots initial condition
#Run for all t
for n in range(nsteps):#Plots real parts                                                                  
    Psi=CrankNicolson(Psi,n)
    Psin.append(Psi)
 #   plt.plot(theta,np.real(T))
    
#plt.show() 

#for T in Tn:#Plots imaginary parts
  #  plt.plot(theta,np.imag(T))

#plt.show()

#for T in Tn:#plots overall average
   # plt.plot(theta,np.abs(T)**2)
   #print(np.sum(np.abs(T)**2)*dtheta)#checking for unitarity

for n in range(0,nsteps,10):
    plt.plot(theta,np.abs(Psin[n])**2)
plt.title('Crank-Nicholson Solve of Quantum System')
plt.xlabel('Theta')
plt.ylabel('|psi^2|')
plt.show()

#Expecatation values

def position(theta,Psi):#calculates expectation value
    TA=0.
    for i in range(isteps):
        TA=TA+theta[i]*(np.abs(Psi[i])**2)*dtheta
    return TA

xbar=np.zeros(nsteps+1)
for n in range(nsteps+1):
    xbar[n]=np.real(position(theta,Psin[n]))
    
#print(xbar[0])
plt.plot(t,xbar)
plt.show()

#angular momentum value, this works as long as Omeg0=1, which it does
Exp_momentum=np.zeros((nsteps+1,2),np.complex)
def momentum(Psi):#calculates expectation value
    P=0.
    for i in range(-1,isteps-1):
        P=P+(1)*(-k)*(Omeg0/ntild)*np.conj(Psi[i])*((Psi[i+1]-Psi[i-1])/(2*dtheta))*dtheta
    return(P)

pbar=np.zeros((nsteps+1),np.complex)
#
for n in range(nsteps+1):
    pbar[n]=np.real(momentum(Psin[n]))
    Exp_momentum[n,0]=t[n]#time
    Exp_momentum[n,1]=momentum(Psin[n])#expectation value of momentum

Exp_momentum=np.matrix(Exp_momentum)#table showing time and expectation value
#print(pbar)
plt.plot(t,pbar)
plt.show()

#print(T)

#poincare surface of sections with expectation values 
V_qm=[]
X_qm=[]

ICsP=int((2*np.pi)/(dt*Omeg1))
Max=nsteps-ICsP

for n in range(nsteps):
    P_Xqm=np.zeros(nsteps)#Max-ICsP)
    P_Vqm=np.zeros(nsteps)#Max-ICsP)
    pbar[n]=np.real(momentum(Psin[n]))
    xbar[n]=np.real(position(theta,Psin[n]))
    V_qm.append(pbar)
    X_qm.append(xbar)
    for k in range(ICsP,Max,ICsP): # turn this loop into the POINCARE SURFACES
        P_Xqm[k-ICsP] = X_qm[n][k]
        P_Vqm[j-ICsP] = V_qm[n][k]
        P_Xqm=((P_Xqm+np.pi)%(2*np.pi))-np.pi
    #plt.plot(P_Xqm,P_Vqm,',')
plt.plot()
plt.show()
#print(len(theta))